In [14]:
%matplotlib inline
%load_ext rpy2.ipython
import pandas as pd
import ggplot

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [15]:
def split_name(d):
    d = d.copy()
    d1 = d.name.str.extract(r'(\d.+)\.(\d+)', expand=True)
    d2 = d1[0].str.split('_', expand=True)
    d['uid'] = d1[1]
    cols = ('coverage', 'universal_efficacy', 'seasonal_efficacy', 'proportion_universal')
    for i,c in zip(range(4), cols):
        d[c] = d2[i].astype(float)
    del d['name']
    return d 

def summarize(d):
    
    def summarize_by_age(d):
        return pd.Series({
            'individuals': d.N_p.mean(),
            'vaccines': d.V_i.sum(),
            'infections': d.IS_i.sum()
        })

    d1 = split_name(d)
    d1['year'] = (d1.day.astype('float') / 360.0).astype('int')
    
    return d1

In [16]:
d = summarize(pd.read_csv('poe_output.csv'))

In [17]:
d.head()

,age,day,N_i,S_i,S_p,E_i,E_p,I_i,I_p,Y_i,...,IS_p,V_i,V_p,N_p,uid,coverage,universal_efficacy,seasonal_efficacy,proportion_universal,year
0,"[65, 106)",999,0,0,199760,0,0,0,0,0,...,0,0,462808,200011,1549524629289121784,0.4,0.3,0.6,1.0,2
1,"[65, 106)",999,0,0,199757,0,0,0,0,0,...,0,0,455602,200011,8050694804863763052,0.4,0.2,0.6,1.0,2
2,"[65, 106)",999,0,0,199747,0,0,0,0,0,...,0,0,402517,200011,3936672713574915758,0.4,0.4,0.6,0.0,2
3,"[65, 106)",999,0,0,199738,0,0,0,0,0,...,0,0,462227,200011,7548959389907938106,0.4,0.4,0.6,1.0,2
4,"[65, 106)",999,0,0,199728,0,0,0,0,0,...,0,0,450574,200011,1942978657812566109,0.4,0.2,0.6,1.0,2


In [18]:
d.to_csv('initial_results.csv',index=False)